<a href="https://www.kaggle.com/code/notsau/cosine-similarity-for-tedx-recommendation?scriptVersionId=161445121" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd

# EDA

In [2]:
df=pd.read_csv("/kaggle/input/ted-talks/transcripts.csv")

In [3]:
df.head()

,transcript,url
0,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...
1,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...
2,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...
3,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...
4,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...


In [4]:
df.shape

(2467, 2)

In [5]:
df["transcript"][0]

'Good morning. How are you?(Laughter)It\'s been great, hasn\'t it? I\'ve been blown away by the whole thing. In fact, I\'m leaving.(Laughter)There have been three themes running through the conference which are relevant to what I want to talk about. One is the extraordinary evidence of human creativity in all of the presentations that we\'ve had and in all of the people here. Just the variety of it and the range of it. The second is that it\'s put us in a place where we have no idea what\'s going to happen, in terms of the future. No idea how this may play out.I have an interest in education. Actually, what I find is everybody has an interest in education. Don\'t you? I find this very interesting. If you\'re at a dinner party, and you say you work in education — Actually, you\'re not often at dinner parties, frankly.(Laughter)If you work in education, you\'re not asked.(Laughter)And you\'re never asked back, curiously. That\'s strange to me. But if you are, and you say to somebody, you

# Feature Enginering & Modelling

In [6]:
df["title"] = df["url"].map(lambda x:x.split("/")[-1])
#url'yi / parçalara böler ve en sondaki parça (genelde başlık olur) onu alıp title sütununa eşitler

In [7]:
df=df[["transcript","title"]]

In [8]:
df.head()

,transcript,title
0,Good morning. How are you?(Laughter)It's been ...,ken_robinson_says_schools_kill_creativity\n
1,"Thank you so much, Chris. And it's truly a gre...",al_gore_on_averting_climate_crisis\n
2,"(Music: ""The Sound of Silence,"" Simon & Garfun...",david_pogue_says_simplicity_sells\n
3,If you're here today — and I'm very happy that...,majora_carter_s_tale_of_urban_renewal\n
4,"About 10 years ago, I took on the task to teac...",hans_rosling_shows_the_best_stats_you_ve_ever_...


In [9]:
import nltk
import re
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
df["transcript"] = df["transcript"].apply(clean)

[nltk_data] Downloading package stopwords to

[nltk_data]     C:\Users\slymn\AppData\Roaming\nltk_data...

[nltk_data]   Package stopwords is already up-to-date!


In [10]:
df.head()

,transcript,title
0,good morn youlaughterit great hasnt ive blown ...,ken_robinson_says_schools_kill_creativity\n
1,thank much chris truli great honor opportun co...,al_gore_on_averting_climate_crisis\n
2,music sound silenc simon garfunkelhello voic ...,david_pogue_says_simplicity_sells\n
3,your today — im happi — youv heard sustain dev...,majora_carter_s_tale_of_urban_renewal\n
4,year ago took task teach global develop swedi...,hans_rosling_shows_the_best_stats_you_ve_ever_...


In [11]:
from sklearn.feature_extraction import text

ted_talks = df["transcript"].tolist()

bi_tfidf = text.TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
bi_matrix = bi_tfidf.fit_transform(ted_talks)

uni_tfidf = text.TfidfVectorizer(stop_words="english")
uni_matrix = uni_tfidf.fit_transform(ted_talks)

from sklearn.metrics.pairwise import cosine_similarity

bi_sim = cosine_similarity(bi_matrix)
uni_sim = cosine_similarity(uni_matrix)

In [12]:
def recommend_ted_talks(x):
    return ".".join(df["title"].loc[x.argsort()[-5:-1]])
    
df["ted_talks_uni"] = [recommend_ted_talks(x) for x in uni_sim]
df["ted_talks_bi"] = [recommend_ted_talks(x) for x in bi_sim]

In [13]:
df['ted_talks_uni']=df['ted_talks_uni'].str.replace("_", " ").str.upper().str.strip().str.split("\n")
df["ted_talks_bi"]=df['ted_talks_bi'].str.replace("_", " ").str.upper().str.strip().str.split("\n")

In [14]:
df.head()

,transcript,title,ted_talks_uni,ted_talks_bi
0,good morn youlaughterit great hasnt ive blown ...,ken_robinson_says_schools_kill_creativity\n,[GEOFFREY CANADA OUR FAILING SCHOOLS ENOUGH IS...,[KANDICE SUMNER HOW AMERICA S PUBLIC SCHOOLS K...
1,thank much chris truli great honor opportun co...,al_gore_on_averting_climate_crisis\n,[ALICE BOWS LARKIN WE RE TOO LATE TO PREVENT C...,[AL GORE THE CASE FOR OPTIMISM ON CLIMATE CHAN...
2,music sound silenc simon garfunkelhello voic ...,david_pogue_says_simplicity_sells\n,[RICARDO SEMLER HOW TO RUN A COMPANY WITH ALMO...,"[KARY MULLIS ON WHAT SCIENTISTS DO, .JOHN MAED..."
3,your today — im happi — youv heard sustain dev...,majora_carter_s_tale_of_urban_renewal\n,[KENT LARSON BRILLIANT DESIGNS TO FIT MORE PEO...,[JEFF SPECK 4 WAYS TO MAKE A CITY MORE WALKABL...
4,year ago took task teach global develop swedi...,hans_rosling_shows_the_best_stats_you_ve_ever_...,"[HANS ROSLING ON GLOBAL POPULATION GROWTH, .HA...","[HANS ROSLING ON GLOBAL POPULATION GROWTH, .HA..."


In [15]:
df["ted_talks_bi"][0]

['KANDICE SUMNER HOW AMERICA S PUBLIC SCHOOLS KEEP KIDS IN POVERTY',
 '.GEOFFREY CANADA OUR FAILING SCHOOLS ENOUGH IS ENOUGH',
 '.SIR KEN ROBINSON BRING ON THE REVOLUTION',
 '.KEN ROBINSON HOW TO ESCAPE EDUCATION S DEATH VALLEY']